In [1]:
import sys
sys.path.append('/Users/ianleefmans/Desktop/stock-sentiment/utils')
from data import Database, ScrapeWSB, Stock
%cd /Users/ianleefmans/Desktop/stock-sentiment

/Users/ianleefmans/Desktop/stock-sentiment


In [2]:
import transformers
import torch
from transformers import BertTokenizer, BertForSequenceClassification

In [37]:

db = Database()
db.use_database('DB1')

print(db.query('show tables;'))

connection established
[(datetime.datetime(2021, 5, 28, 16, 21, 30),)]
[('COMMENTS',), ('POSTS',), ('STOCKS',)]


In [17]:
print(db.query('''SELECT * FROM STOCKS ;'''))

[('GME', datetime.datetime(2021, 5, 14, 1, 43, 41))]


In [ ]:
indexes = db.query('''SELECT POST_ID FROM POSTS;''')

In [39]:
db.query("SELECT TITLE FROM POSTS WHERE POST_ID='{}' AND STOCK_ID='{}'".format(indexes[0][0], 'GME'))[0][0]

"I, too, YOLO'D my retirement account into GME when I switched jobs. PICK ME UP ON THE WAY TO THE MOO"

In [45]:
indices = db.query("SELECT POST_ID FROM POSTS WHERE StOCK_ID='{}';".format("GME"))

In [62]:
db.query("SELECT COUNT(*) FROM POSTS WHERE StOCK_ID='{}';".format("GME"))[0][0]

10

In [8]:
import torch
from torch.utils.data import DataLoader
from datahelper import PostDataset, get_indices
from models import SentimentModel
import yaml

def get_config():
    with open("config.yml") as file:
        config = yaml.load(file, Loader=yaml.FullLoader)
    return config['Inference']

indices = get_indices('GME', inference=True)

config = get_config()

post_data = PostDataset(config['embedding_max_len'], indices['post_ids'])

post_dataloader = DataLoader(dataset=post_data, 
                        batch_size=config['batch_size'], 
                        num_workers=config['num_workers'],
                        shuffle=config['shuffle']
                       )

connection established
[(datetime.datetime(2021, 5, 28, 19, 15, 26),)]
connection established
[(datetime.datetime(2021, 5, 28, 19, 15, 27),)]


In [9]:
sample = iter(post_dataloader).next()

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [12]:
sample

{'post': ['Bought more GME cuz I’m retarded.',
  "I, too, YOLO'D my retirement account into GME when I switched jobs. PICK ME UP ON THE WAY TO THE MOO",
  '(O.C.) Keep shorting GME 🤡',
  'GME Great investment! Bought 600 more shares on my second account. 1600 shares in total! Life is gre',
  'Reviewing my GME investment'],
 'post_input_ids': tensor([[  101,  9326, 11266,  1167, 14748,  2036, 16408,  1584,   146,   787,
            182,  1231,  6817,  4902,   119,   102,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0, 

In [16]:
sample['post_attention_mask'].size()

torch.Size([5, 200])

In [55]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

inputs = tokenizer(["Hello, my dog is cute", "Hello, my dog is cute"], return_tensors="pt")
labels = torch.tensor([1,1]).unsqueeze(0)  # Batch size 1
outputs = model(**inputs, labels=labels)
loss = outputs.loss
logits = outputs.logits

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [56]:
logits

tensor([[-0.2388, -0.3600],
        [-0.2388, -0.3600]], grad_fn=<AddmmBackward>)

In [58]:
loss

tensor(0.7556, grad_fn=<NllLossBackward>)

In [57]:
softmax = torch.nn.Softmax(dim=1)
softmax(logits)

tensor([[0.5303, 0.4697],
        [0.5303, 0.4697]], grad_fn=<SoftmaxBackward>)

In [52]:
logits

tensor([[0.2457, 0.3727]], grad_fn=<AddmmBackward>)

In [54]:
loss

tensor(0.6317, grad_fn=<NllLossBackward>)

In [59]:
softmax = torch.nn.Softmax(dim=1)
probs = softmax(logits)

In [33]:
softmax = torch.nn.Softmax(dim=1)
softmax(logits)

tensor([[0.4012, 0.5988],
        [0.4012, 0.5988]], grad_fn=<SoftmaxBackward>)

In [24]:
inputs

{'input_ids': tensor([[  101,  7592,  1010,  2026,  3899,  2003, 10140,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}

In [32]:
inputs['attention_mask'].size()

torch.Size([2, 8])

In [45]:
_, preds = torch.max(logits, dim=1)

In [39]:
preds

tensor([1])

In [46]:
preds

tensor([0, 0])

In [65]:
probs

tensor([[0.5303, 0.4697],
        [0.5303, 0.4697]], grad_fn=<SoftmaxBackward>)

In [63]:
probs.sum(dim=0)

tensor([1.0605, 0.9395], grad_fn=<SumBackward1>)

In [66]:
probs.mean(dim=0)

torch.Size([2])

In [75]:
x = torch.zeros([2])

In [77]:
x += probs.mean(dim=0)

In [78]:
x

tensor([0.5303, 0.4697], grad_fn=<AddBackward0>)

In [79]:
x/2

tensor([0.2651, 0.2349], grad_fn=<DivBackward0>)

In [80]:
len(post_dataloader)

2